# **Scraping filmów z serwisu Filmweb**

# 1\. Wprowadzenie

W moim projekcie zajmę się analizą preferencji użytkowników serwisu **Filmweb**. Chcę poznać jakie parametry mają decydujący wpływ na końcową ocenę filmu. Aby przeprowadzić takie badanie należy pozyskać odpowiednie dane, czyli informację o filmach z możliwie wieloma informacjami dostępnymi na stronie. Wśród tych danych można wyróżnić te za które odpowiadają użytkownicy serwisu i zależą od ich preferencji oraz aktywości jak na przykład średnia ocena, liczba ocen i liczba wątków na forum, oraz dane o filmie jak kraj, gatunek lub reżyser. Dodatkowo w serwisie występuje grupa krytyków filmowych, których recenzje są wyróżnice, a liczba ocen oraz średnia jest liczona oddzielnie.

Aby uzyskać aktualne i możliwie kompletne dane wykorzystam scraping, czyli automatyczny proces przeszukiwania i pobierania informacji z Internetu. Dzięki samodzielnemu napisaniu służącego temu skryptu mogę zebrać wszystkie dane potencjalnie przydatne do analizy i następnie ewentualnie odrzucić je na dalszym etapie. Web scraping musi być poprzedzony zapoznaniem się ze strukturą strony, po której skrypt ma się poruszać. Należy poznać budowę i adresy podstron interesujących nas, znaleźć sprawny system nawigacji pomiędzy kolejnymi elementami oraz określić gdzie we fragmentach kodu HTML, CSS czy JavaScript znajdują się dane, które chcemy pobrać.

Należy również pamiętać o tym że strony internetowe są na bieżąco rozwijane i w trakcie tworzenia lub używania algorytmu mogą zajść na nich zmiany wymuszające dostosowanie kodu. Duża aktualizacja może wręcz wymusić całkowite przepisanie skryptu. Problem ten pojawił się również podczas tego projektu, gdzie zmianie uległa struktura strony wyszukiwarki, używanej do poruszania się pomiędzy kolejnymi filmami. Dokładny opis zaistniałej sytuacji zawarłam poniżej.

## 1.1 Przygotowanie środowiska

Na początku deklaruję wykorzystywane biblioteki. Dane z HTML będę pobierać za pomocą *requests* i następnie parsować je przy użyciu *BeautifulSoup*. Do zbierania danych w macierzy będzie służyła biblioteka numpy, aby przerobić je na DataFrame i wyeksportować do pliku CSV biblioteka *pandas*, a *random* oraz *time* posłużą odczekiwaniu między kolejnymi zapytaniami, aby uniknąć potencjalnego zablokowania przez serwis Filmweb.

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from random import random
import time

## 1.2 Poszukiwanie metody scrapowania Filmwebu

Na początku należy przeanalizować możliwość scrapowania strony **filmweb.pl**. Strona z pojedynczym filmem ma adres jak poniżej:

https://www.filmweb.pl/film/Sherlock+Holmes-2009-426062

Widać, zatem że składa się on z 3 zmiennych, tytułu, roku premiery i ID filmu. Przechodzenie więc do filmów na przykład na podstawie losowania ID jest więc niemożliwe, ponieważ nie będę znać tytułu oraz roku. Należy więc odwiedzać kolejne strony poprzez przeszukiwanie zbiorczych stron z filmami. Pierwszą możliwością byłoby przeglądanie rankingu najlepszych filmów:

https://www.filmweb.pl/ranking/film

Jednakże na tej stronie jest tylko 500 najlepszych pozycji. Rankingi dla pojedynczych gatunków zwracają jeszcze mniej, bo jedynie 100 filmów, przykładowo:

https://www.filmweb.pl/ranking/film/Akcja/28

Aby móc dostać się do wszystkich dostępnych filmów, należy skorzystać z wyszukiwarki:

https://www.filmweb.pl/films/search

Pozwala ona na filtrowanie filmów po gatunkach, krajach, latach produkcji czy ocenach. Decyduję się, że będę przeszukiwała kolejne gatunki filmów w kolejności malejącej liczby ocen. W ten sposób przechodząc przez wszystkie istniejące w serwisie gatunki, zbiorę dane dotyczące wszystkich filmów. Najpierw należy jednak zdobyć listę wszystkich gatunków oraz numerów, które im odpowiadają w linku, jak przykładowo 28 dla filmów Akcji:

https://www.filmweb.pl/search#/films?genres=28

In [2]:
# scrapowanie numerów odpowiadających gatunkom na Filmwebie z wyszukiwarki
URL = "https://www.filmweb.pl/ranking/film"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
genres_labels = soup.find_all("label", class_="filterSelect__option fwBtn fwBtn--checkbox")

genres_list = []
numbers_list = []

for genre in genres_labels:
    genre_name = genre["data-name"]
    number = genre["data-value"]
    genres_list.append(genre_name)
    numbers_list.append(number)

print(genres_list)
print()
print(numbers_list)

['Biograficzny', 'Dla dzieci', 'Dokumentalny', 'Dramat', 'Familijny', 'Surrealistyczny', 'Historyczny', 'Kostiumowy', 'Melodramat', 'Musical', 'Obyczajowy', 'Przygodowy', 'Sensacyjny', 'Western', 'Film-Noir', 'Komedia obycz.', 'Romans', 'Dramat obyczajowy', 'Psychologiczny', 'Satyra', 'Katastroficzny', 'Baśń', 'Polityczny', 'Muzyczny', 'Dreszczowiec', 'Czarna komedia', 'Krótkometrażowy', 'Religijny', 'Gangsterski', 'Biblijny', 'Dokumentalizowany', 'Komedia kryminalna', 'Dramat historyczny', 'Groteska filmowa', 'Sportowy', 'Poetycki', 'Szpiegowski', 'Dramat sądowy', 'Anime', 'Niemy', 'Fabularyzowany dok.', 'XXX', 'Sztuki walki', 'Przyrodniczy', 'Propagandowy', 'Animacja dla dorosłych', 'True crime', 'Afganistan', 'Albania', 'Algieria', 'Andora', 'Angola', 'Antigua i Barbuda', 'Arabia Saudyjska', 'Argentyna', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Autonomia Palestyńska', 'Azerbejdżan', 'Bahamy', 'Bahrajn', 'Bangladesz', 'Barbados', 'Belgia', 'Belize', 'Benin', 'Bhutan', 'Białoruś',

Przeszukując wszystkie opcje filtrowania filmów znalazłam, że są to kolejno gatunki, kraje produkcji oraz lata. Poniżej nazw filtrów wyświetliłam odpowiadające im numery, które pojawiają się w adresach stron. Ostatnim gatunkiem powinien być *True crime*, ponieważ dalej jest już pierwszy kraj, czyli *Afganistan*. Znajduję zatem na którym miejscu w liście jest ten gatunek i dla sprawdzenia wyświetlam wszystkie otrzymane gatunki. Do późniejszego użytku zapisuję zmienną *genres*, aby wiedzieć ile różnych gatunków jest w serwisie.

In [4]:
genres = genres_list.index("True crime") + 1
for i in range(genres):
    print (genres_list[i], end="; ")

Biograficzny; Dla dzieci; Dokumentalny; Dramat; Familijny; Surrealistyczny; Historyczny; Kostiumowy; Melodramat; Musical; Obyczajowy; Przygodowy; Sensacyjny; Western; Film-Noir; Komedia obycz.; Romans; Dramat obyczajowy; Psychologiczny; Satyra; Katastroficzny; Baśń; Polityczny; Muzyczny; Dreszczowiec; Czarna komedia; Krótkometrażowy; Religijny; Gangsterski; Biblijny; Dokumentalizowany; Komedia kryminalna; Dramat historyczny; Groteska filmowa; Sportowy; Poetycki; Szpiegowski; Dramat sądowy; Anime; Niemy; Fabularyzowany dok.; XXX; Sztuki walki; Przyrodniczy; Propagandowy; Animacja dla dorosłych; True crime; 

# 2\. Definiowanie funkcji

Do przyszłego użytku definiuję funkcję **find_between** zwracającą string pomiędzy dwoma innymi stringami. Będzie ona przydatna przy scrapowaniu, ponieważ nie do wszystkich wiadomości można się dostać poprzez tagi HTML i CSS. Niektóre będę wyciągać z fragmentów adresów stron lub skryptów JavaScript widząc miedzy jakimi znakami się znajdują.

In [ ]:
def find_between(s, first, last):
    try:
        start = s.index(first) + len(first)
        end = s.index(last, start)
        return s[start:end]
    except ValueError:
        return ""

Głównym elementem kodu jest funkcja **filmweb_scraping**, która przyjmuje parametry *start_page* i *pages*, które odpowiadają za początkową stronę od której zacznę scraping i liczbę kolejnych stron do scrapowania. Dzięki temu mogę scrapować dane w kawałkach i co pewien czas eksportować do pliku CSV. Funckja bazuje na dwóch pętlach **for**, które iteruję po wszystkich gatunkach i po podanej przez nas liczbie stron w każdym z gatunków. W zmiennej *columns* zebrałam nagłówki dla przyszłego DataFrame, liczba tych nagłówków definiuje rozmiar macierzy *data*, do której będę dopisywać kolejne wiersze. W tym sposobie przeszukiwania serwisu natrafię na duplikaty, ponieważ film może mieć przypisanych kilka gatunków i wtedy pojawia się w wyszukiwarce w każdym z nich. Aby skrócić czas zbierania danych pobieram podczas scrapingu dane z pliku *filmweb.csv* do której eksportuję dane dotyczące filmów, aby sprawdzać czy dany film nie został już pobrany, na podstawie unikatowego ID. Sprawdzam również, czy film nie jest obecny w aktualnie tworzonej macierzy *data*. Używając pętli **for** z iteratorami *i* i *j* będę otwierać strony według poniższego f-stringu:

`f"https://www.filmweb.pl/films/search?genres={numbers_list[i]}&orderBy=count&descending=true&page={j}"`

Numery gatunków biorę z wcześniejszej listy *numbers_list*. Zawarcie w linku "&orderBy=count&descending=true" zapewnie że filmy będą posortowane według liczby ocen w kolejności malejącej. Ponieważ będę później skupiać się na analizie jak użytkownicy oceniają filmy, zakładam że film musi posiadać minimum 500 ocen, aby jego ocena była wiarygodna. Pozostałe filmy pomijam. Dodając do linku "&page=" otrzymuję konkretną stronę zawierającą 10 kolejnych filmów.

***
**UWAGA!**

Od czasu wykonywania scrapingu (grudzień 2022) uległ zmianie format linków w wyszukiwarce oraz sposób generowania strony. Obecnie lista filmów w wyszukiwarce wyświetlana jest trybie infinite scroll i aby pobierać kolejne film należy wejść z nią w interakcję, na przykład za pomocą biblioteki *selenium*. Nowy link ma schemat:

https://www.filmweb.pl/search#/films?genres=28&orderBy=count.desc&page=1

Użycie fragmentu "&page=" nie wymusza jednak paginacji stron, a jedynie ogranicza ładowanie do 30 kolejnych filmów. Nadal cała interesująca nas zawartość generowana jest za pomocą JavaScriptu i nie da się jej pobrać w sposób jak wcześniej używając jedynie *requests* i *BeautifulSoup*. Wygląd poprzedniej strony wyszukiwarki z paginacją co 10 filmów można zobaczyć w serwisie The Wayback Machine:

https://web.archive.org/web/20221223212507/https://www.filmweb.pl/films/search

Aby dostosować algorytm do aktualnej budowy strony należałoby zmienić jedynie sposób przeszukiwania linków do kolejnych filmów. Dalsze działania, już na stronie filmu i stronie forum dotyczącego filmu, pozostają identyczne, ponieważ tam aktualnie (luty 2023) struktura kodu pozostała bez zmian.
***

Będąc na stronie wyszukiwarki znajduję zawarte na niej 10 filmów i tworzę z nich listę *results*, po której wykonuję kolejną iterację pętlą **for**. Na stronie wyszukiwarki są linki do każdego z 10 filmów, więc przechodzę do nich i parsuje je biblioteką *BeautifulSoup*, następnie przeszukując odpowiednie divy, spany, linki, nagłówki i skrypty i wyciągając z nich wszystkie interesujące mnie dane wymienione w liście *columns*. Lokalizacja potrzebnych informacji została znaleziona poprzez analizę budowy strony oraz metodą prób i błędów uruchamiając scraping aby znaleźć, które elementy nie pojawiają się dla każdego filmu jak budżet, scenarzysta, itp. Na koniec aby poznać liczbę wątków i odpowiedzi na wątki w forum dotyczącym filmu przechodzę pętlą **while** przez wszystkie strony forum sumując wątki i odpowiedzi. Pomiędzy kolejnymi filmami oraz stronami forum odczekuję losowy czas funckją **time.sleep** w połączeniu z **random**.

In [5]:
columns = ["ID","Tytuł","Tytuł org.","Rok","Długość","Ocena","Liczba ocen","Najlepsza ocena","Najgorsza ocena",\
            "Chce zobaczyć","Ocena krytyków","Liczba ocen krytyków","Reżyser","Scenariusz","Gatunek","Kraj","Box Office","Budżet",\
            "1","2","3","4","5","6","7","8","9","10","Wątki","Odpowiedzi"]

def filmweb_scraping(start_page, pages):
    # pobieranie aktualnego pliku csv do pomijania duplikatów
    existing_data = pd.read_csv('filmweb.csv', encoding='utf-8', header=None, sep=';')

    # tworzenie macierzy o wymiarze 1 x liczba kolumn do zbierania danych
    data = np.array([[0]*len(columns)])
    # iterator pomocniczy do wyświetlania numeru aktualnie pobieranego filmu
    iter = 0

    # pętla po wszystkich gatunkach
    for i in range(genres):
        less_than_500_count = False # zmienna zapisująca czy doszłam do filmów poniżej 500 ocen w danym gatunku
        
        # pętla po kilku (pages) stronach najpopularniejszych
        for j in range(start_page, pages + start_page):
            
            # przejście do kolejnego gatunku jeżeli w aktualnym gatunku kolejne filmy mają mniej niż 500 ocen
            if less_than_500_count == True:
                break
            
            # otwieranie kolejnych linków filmami w danym gatunku według liczby ocen malejąco
            URL = f"https://www.filmweb.pl/films/search?genres={numbers_list[i]}&orderBy=count&descending=true&page={j}"
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            # wyciąganie 10 filmów ze strony
            results = soup.find_all("div",class_="preview previewCard previewFilm PreviewFilm")
            # przejście do kolejnego gatunku jeżeli strona jest pusta
            if results == None:
                break

            for result in results:                
                iter += 1
                
                # szukanie linku do filmu i otwieranie go jako nowy request
                link_element = result.find("div", class_="poster__wrapper")
                link = link_element.find("a")
                film_url = "https://www.filmweb.pl" +link["href"]
                film_page = requests.get(film_url)
                film_soup = BeautifulSoup(film_page.content, "html.parser")
                
                # pobieranie danych tytuł, rok, długość
                title_data = film_soup.find("div", class_="filmCoverSection__fP film")
                id = title_data["data-film-id"]
                title = title_data.find("h1").text
                year = title_data.find("div", class_="filmCoverSection__year").text
                
                # sprawdzanie czy film nie jest już w filmweb.csv (pandas - trzeba wybrać '.values') 
                # aktualnym zbiorze (numpy - konflikt porówywania, trzeba wymusić typ int dla obu stron)
                if int(id) in existing_data.iloc[:,0].values or int(id) in data[:,0].astype(int):
                    print(f"{iter}. {title} ({genres_list[i]}) - duplikat!")
                    continue
                
                # sprawdzanie czy premiera przed 2023 + wyświetlanie aktualnie przetwarzanego filmu
                elif int(year) > 2022:
                    print(f"{iter}. {title} ({genres_list[i]}) - przed premierą!")
                    continue
                else:
                    print(f"{iter}. {title} ({genres_list[i]})")

                # original_title może nie być, wtedy jest jak title                    
                original_title_div = title_data.find("div", class_="filmCoverSection__originalTitle")
                if original_title_div == None:
                    original_title = title
                else:
                    original_title = original_title_div.text
                    
                # pobieranie długości filmu
                try:
                    watch_time_str = title_data.find("div", class_="filmCoverSection__duration")
                    watch_time = watch_time_str["data-duration"]
                except:
                    watch_time = "N/A"
                
                # pobieranie danych dot. ocen, jeżeli mniej niż 500 pomijam kolejne filmy w tym gatunku
                try:
                    rate_data = film_soup.find("div", class_="page__container filmCoverSection__ratings afterPremiere")
                    rate_str = rate_data.find("div", class_="filmRating filmRating--hasPanel")
                    rate = rate_str["data-rate"]
                    rating_count = rate_str["data-count"]
                    if int(rating_count) < 500:
                        less_than_500_count = True
                        break
                except:
                    less_than_500_count = True
                    break
                
                # pobieranie najlepszej, najgorszej oceny i ile osób chce zobaczyć film
                best_rating = rate_data.find("span", itemprop="bestRating").text
                worst_rating = rate_data.find("span", itemprop="worstRating").text
                want_to_see_str = rate_data.find_all("div", class_="filmRating__rate")
                want_to_see1 = want_to_see_str[1].text
                want_to_see = int(''.join(filter(str.isdigit, want_to_see1))) 
                
                # pobieranie ocen krytyków
                try:
                    critics = str(rate_data.find("script"))
                    critics_rate = find_between(critics,'"criticRatingData",{rate:',",")
                    critics_rating_count = find_between(critics,',count:',"}")
                except:
                    critics_rate = "N/A"
                    critics_rating_count = 0

                # pobieranie pozostałych danych
                try:
                    director = film_soup.find("div", {"data-type": "directing-info"}).text
                except:
                    director = "N/A"
                try:
                    screenwriting = film_soup.find("div", {"data-type": "screenwriting-info"}).text
                except:
                    screenwriting = "N/A"
                try:
                    boxoffice_str = film_soup.find("div", class_="boxoffice").text
                    boxoffice = int("".join(find_between(boxoffice_str,"$"," w USA").split()))
                except:
                    boxoffice = "N/A"
                    
                # pobieranie gatunku i kraju
                genre_div = film_soup.find("div", itemprop="genre")
                genre = genre_div.find("a").text
                country_div = film_soup.find("div", class_="filmPosterSection__info filmInfo")
                country_str = find_between(str(country_div),'produkcja</h3><div class="filmInfo__info"><span> <a href="/ranking/film/',"/")
                country = country_str.replace("+"," ") # w linku jest + zamiast spacji

                try:
                    budget_div = film_soup.find("div", class_="filmOtherInfoSection__group")
                    budget = int("".join(find_between(str(budget_div),'<div class="filmInfo__header">budżet</div><div class="filmInfo__info">$',"</").split()))  
                except:                
                    budget="N/A"
                
                film_data = [id,title,original_title,year,watch_time,rate,rating_count,best_rating,
                                    worst_rating,want_to_see,critics_rate,critics_rating_count,
                                    director,screenwriting,genre,country,boxoffice,budget]

                # znalezienie skryptu wyświetlającego ile film dostał ocen 1, 2, ..., 10 i dopisanie do film_data
                for item in film_soup.find_all("script"):
                    if '"filmRating"' in item.text:
                        for rating in range(10):
                            film_data = np.append(film_data,(find_between(item.text,f'",countVote{rating + 1}:"','"')))
                            
                # szukanie linku do forum i otwieranie go jako nowy request
                discussion_url = film_url + "/discussion"
                discussion_page = requests.get(discussion_url)
                discussion_soup = BeautifulSoup(discussion_page.content, "html.parser")

                reply_count = 0
                threat_count = 0

                next_page = discussion_soup.find("a", title="następna")

                while next_page is not None:
                    comments = discussion_soup.find_all("span", class_="forumSection__commentsCount")
                    threat_count = threat_count + len(discussion_soup.find_all("div", class_="forumSection__topRow"))
                    for comment in comments:
                        reply_count = reply_count + int(comment.text)
                    discussion_page = requests.get("https://www.filmweb.pl" + next_page["href"])
                    discussion_soup = BeautifulSoup(discussion_page.content, "html.parser")
                    next_page = discussion_soup.find("a", title="następna")
                    time.sleep(random()/20)
                else:
                    try:
                        comments = discussion_soup.find_all("span", class_="forumSection__commentsCount")
                        threat_count = threat_count + len(discussion_soup.find_all("div", class_="forumSection__topRow"))
                        for comment in comments:
                            reply_count = reply_count + int(comment.text)
                    except:
                        pass
                
                # dodanie danych o wątkach i odpowiedziach
                film_data = np.append(film_data,[threat_count,reply_count])           

                # dodawanie danych do listy jako kolejny wiersz
                data = np.append(data,[film_data],axis=0)
                                                
                # czyszczenie zmiennych
                del id,title,original_title,year,watch_time,rate,rating_count,best_rating,\
                    worst_rating,want_to_see,critics_rate,critics_rating_count,\
                    director,screenwriting,genre,country,boxoffice,budget,threat_count,reply_count

                time.sleep(random()/10)
    
    # zwrócenie macierzy numpy 
    return data

Poniższa funkcja **data_to_dataframe** służy przejścia z macierzy *numpy* na DataFrame *pandas*. Dodatkowo usuwam pierwszy wiersz, które wcześniej wypełniłam zerami, aby zainicjować macierz w odpowiednim rozmiarze. Przypisuję również nazwy kolumn ze zmiennej *columns*.

Funckja **update_csv** dopisuje do naszego pliku zbiorczego kolejne wiersze bez nagłówków z ustalonym separatorem ";" który nie pojawia się w przeciwieństwie do "," żadnych danych.

Funckja **backup_csv** służy do odczytu aktulnego pliku zbiorczego CSV i zapisu go do pliku CSV w którym dodaję nagłówki i aktualny czas do nazwy pliku, aby dostać unikatowy plik kopii zapasowej wywołując ją co pewien czas w głównej pętli.

In [6]:
def data_to_dataframe(data):
    if data[0, 0] == "0":
        data = np.delete(data, obj=0, axis=0)
    df = pd.DataFrame(data=data[0:, 0:], index=[i + 1 for i in range(data.shape[0])], columns=columns)
    return df

def update_csv(df):
    df.to_csv("filmweb.csv", mode="a", encoding="utf-8", index=False, header=False, sep=";")
    
def backup_csv():
    df = pd.read_csv("filmweb.csv", encoding="utf-8", header=None, names=columns, sep=";")
    df.drop_duplicates(subset=["ID"], inplace=True)
    df.to_csv(
        f'filmweb_{time.strftime("%Y%m%d-%H%M%S")}.csv',
        encoding="utf-8",
        index=False,
        header=columns,
        sep=";",
    )

Poniżej znajduje się główna pętla programu, która przechodząc przez strony 1-999 co *pages*=10 stron zbiera dane za pomocą funkcji **filmweb_scraping**. Po wykonaniu tej funkcji dane zamieniane na DataFrame poprzez **data_to_dataframe** oraz zapisywane do pliku CSV poprzez **update_csv**. Dodatkowo co 50 stron zapisuję dane do osobnego pliku używając **backup_csv**. Za pomocą **try** i **except** zbieram informacje na temat ewentualnych błędów w scrapingu i miejsca wystąpienia błędu, co pozwala na określenie miejsca do którego informacje zostały pobrane poprawnie i ponowne puszczenie funkcji od wskazanej strony po dostosowaniu kodu. Na koniec wczytuję ukończony plik CSV i zapisuję go z nagłówkami *columns*.

In [8]:
pages = 10
for start_page in range (1, 999, pages):
    try:
        data = filmweb_scraping(start_page, pages)
        df = data_to_dataframe(data)
        update_csv(df)
        #zapis do nowego pliku co 50 stronę
        if ((start_page > 1) & (start_page % 50 == 1)):
            backup_csv()
    except:
        print(f"Błąd na stronie nr {start_page}")

df = pd.read_csv("filmweb.csv", encoding="utf-8", header=None, names=columns, sep=";")
df.to_csv('filmweb.csv', encoding="utf-8", index=False, header=columns, sep=";")

1. The Man in the Wolf Mask (Biograficzny)
2. Zarifa Ghafari: Wszystko w jej rękach (Dokumentalny)
3. Ballada o Narayamie (Dramat) - duplikat!
4. Wiem, że na imię mam Steven (Dramat) - duplikat!
5. Szkoda twoich łez (Dramat) - duplikat!
6. Ugór (Dramat) - duplikat!
7. 90 minutter (Dramat) - duplikat!
8. To my, a to ja (Dramat) - duplikat!
9. Prawdziwe kobiety są zaokrąglone (Dramat) - duplikat!
10. Poongsan (Dramat) - duplikat!
11. Sala numer 6 (Dramat) - duplikat!
12. Dlaczego Pan R. oszalał? (Dramat) - duplikat!
13. Sierota (Dramat) - duplikat!
14. Stalin (Dramat) - duplikat!
15. Mój koniec. Twój początek (Dramat) - duplikat!
16. Ciotka Hitlera (Dramat) - duplikat!
17. Szczury (Dramat) - duplikat!
18. Lick the Star (Dramat) - duplikat!
19. Pod oliwkami (Dramat) - duplikat!
20. Ali się żeni (Dramat) - duplikat!
21. The Bad Mother's Handbook (Dramat) - duplikat!
22. Mój problem to ty (Dramat) - duplikat!
23. Uczniowie zmieniają świat (Dramat) - duplikat!
24. Szczęście na raty (Dramat) 